# Assignment - Time Series Analysis of London Energy Consumption

KATE expects your code to define variables with specific names that correspond to certain things we are interested in.

KATE will run your notebook from top to bottom and check the latest value of those variables, so make sure you don't overwrite them.

* Remember to uncomment the line assigning the variable to your answer and don't change the variable or function names.
* Use copies of the original or previous DataFrames to make sure you do not overwrite them by mistake.

You will find instructions below about how to define each variable.

Once you're happy with your code, upload your notebook to KATE to check your feedback.

A time series from smart-meters in London is provided from end of 2011 to 2014. The data contains the daily consumption (kWh) averaged over several households in London. 

The last timestamp for which consumpton is available is January 31st 2014. 

The aim of this assignment is to build a forecasting model that predicts the consumption for February 2014 (exluding 28th, so from the 1st to the 27th).

## Get the data
Load the dataset using `.read_csv()` and assign it to a DataFrame, `df`:

In [87]:
import pandas as pd

df = pd.read_csv("data/london_smartmeter_basic.csv")

df.head()

,day,consumption,evaluation_set
0,2011-11-24,7.7580,False
1,2011-11-25,8.7385,False
2,2011-11-26,9.4770,False
3,2011-11-27,9.8030,False
4,2011-11-28,7.0690,False


## Get Training and Evaluation Time Series

Separate the original dataframe into a training set `ts` and evaluation set `ts_eval`.

`ts_eval` is the evaluation time series, it contains the list of days you will need to predict consumption for so KATE can evaluate the performance of your model. 

In the dataframe provided, the column `evaluation_set` determines whether a row is for evaluation or not.

Get all the rows for the evaluation set using:

```
df.loc[df.evaluation_set]
```

In [88]:
def preprocess(df):
    """This function takes a dataframe and preprocesses it so it is
    ready for the training stage.

    The DataFrame contains the time axis and the target column.

    It also contains some rows for which the target column is unknown.
    Those are the observations you will need to predict for KATE
    to evaluate the performance of your model.

    Here you will need to return the training time series: ts together
    with the preprocessed evaluation time serie: ts_eval.

    Make sure you return ts_eval separately! It needs to contain
    all the rows for evaluation -- they are marked with the column
    evaluation_set. You can easily select them with pandas:

         - df.loc[df.evaluation_set]


    :param df: the dataset
    :type df: pd.DataFrame
    :return: ts, ts_eval
    """
    
    # set as date time 
    df["day"] = pd.to_datetime(df["day"], format = "%Y-%m-%d") 
    
    # rename columns
    df.rename(columns = {"day":"ds", "consumption":"y"}, inplace = True)
    
    ts = df.loc[df["evaluation_set"] == False]    
    ts_eval = df.loc[df["evaluation_set"] == True]
    
    ts = ts.drop("evaluation_set", axis = 1)
    ts_eval = ts_eval.drop("evaluation_set", axis = 1)
    
    return ts, ts_eval

    
    
#raise NotImplementedError

Once you have implemented the `preprocess` function, you can call it with `ts, ts_eval = preprocess(df)`:

In [89]:
# Your code here...
ts, ts_eval = preprocess(df)

In [93]:
len(ts_eval)

27

## Train an AR model

Write a function to train an AR model using your train time series `ts`.

You can use the  Autoregressive AR(p) model `tsa.AR` from the `statsmodels` module to train an AR model. Another option is to use the module Prophet; do not forget to set the growth and the seasonality parameters.

*NOTE*: Since with this project your model will be trained directly on KATE, it is limited to models that can be trained under 1min. You will receive a `TimeoutError` if your model takes too long.

In [94]:
from prophet import Prophet

def train(ts):
    """Trains a new model on ts and returns it.

    :param ts: your processed training time serie
    :type ts: pd.DataFrame
    :return: a trained model
    """
    
    model = Prophet()
    model.fit(ts)
    
    return model
    
#raise NotImplementedError

Once you have implemented the `train` function, you can call with with `model = train(ts)`:

In [95]:
# Your code here...
model = train(ts)


17:59:31 - cmdstanpy - INFO - Chain [1] start processing
17:59:31 - cmdstanpy - INFO - Chain [1] done processing


## Evaluate Your Model

Write a function `predict` that takes the model you have trained as well as a test time series (on KATE this will be the `ts_eval` that you processed above, but you can test this function locally with your own time series). 

The function should return the predictions on the test set assigned to a variable called `y_pred`.


In [114]:
def predict(model, ts_test):
    """This functions takes your trained model as well
    as a processed test time series and returns predictions.

    This should return your predictions either as a pd.DataFrame with one column
    or a pd.Series

    :param model: your trained model
    :param ts_test: a processed test time serie (on KATE it will be ts_eval)
    :return: y_pred, your predictions
    """
    
    # Create future df
    future_df = model.make_future_dataframe(periods=27, include_history=False)
    
    #predictions for future dates created above
    forecast = model.predict(future_df)
    
    return forecast["yhat"]
    

Once implemented, you can check our your predictions by calling predict with `model` and `ts_eval`:

In [115]:
# Your code here...
y_pred = predict(model, ts_eval)

In [116]:
y_pred

0     9.042186
1     9.448268
2     8.860092
3     8.735572
4     8.648586
5     8.598180
6     8.605243
7     8.865430
8     9.258809
9     8.659174
10    8.524474
11    8.428599
12    8.370589
13    8.371304
14    8.626356
15    9.015760
16    8.413240
17    8.276674
18    8.179869
19    8.121784
20    8.123195
21    8.379631
22    8.771024
23    8.171020
24    8.037420
25    7.943964
26    7.889540
Name: yhat, dtype: float64